<a href="https://colab.research.google.com/github/lishura/Apache_Spark/blob/main/HW2_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Условие: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2ccebc2253c87db5081bdabe24a33cbe720f9cf47ec928206bc913f5f7b2e969
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [7]:
spark = SparkSession.builder.appName('salary').getOrCreate()
data = [(1, 'Joe', 70, 3), (2, 'Henry', 80, 4), (3, 'Sam', 60, 'Null'), (4, 'Max', 90, 'Null') ]
df = spark.createDataFrame(data, ['id', 'name', 'salary', 'managerId'])
df.show()

+---+-----+------+---------+
| id| name|salary|managerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



In [10]:
df_managers = df.select('id', 'salary').withColumnRenamed('id', 'manager_id').withColumnRenamed('salary', 'manager_salary')
df_managers.show()

+----------+--------------+
|manager_id|manager_salary|
+----------+--------------+
|         1|            70|
|         2|            80|
|         3|            60|
|         4|            90|
+----------+--------------+



In [11]:
df_joined = df.join(df_managers, df['managerId'] == df_managers['manager_id'], 'inner')
df_joined.show()

+---+-----+------+---------+----------+--------------+
| id| name|salary|managerId|manager_id|manager_salary|
+---+-----+------+---------+----------+--------------+
|  1|  Joe|    70|        3|         3|            60|
|  2|Henry|    80|        4|         4|            90|
+---+-----+------+---------+----------+--------------+



In [13]:
df_joined.filter(df_joined['salary'] > df_joined['manager_salary']).select(df_joined.name).show()

+----+
|name|
+----+
| Joe|
+----+

